#Databricks notebook: Bronze - Risco de Fogo(Diário)

In [0]:
# ================================================================
# Bronze - Importar arquivo NetCDF (INPE Fire Risk)
# ================================================================
from pyspark.sql import SparkSession, functions as F
import os

spark = SparkSession.builder.appName("Bronze_FireRisk_NetCDF").getOrCreate()

In [0]:
# ================================================================
# Parâmetro recebido via Job Databricks
# ================================================================
dbutils.widgets.text("data_ref_carga", "")
data_ref_carga = dbutils.widgets.get("data_ref_carga")

if not data_ref_carga:
    raise ValueError("❌ Parâmetro 'data_ref_carga' não informado (formato esperado: yyyy-MM-dd)")

print(f"🗓️ Data de referência da carga: {data_ref_carga}")

In [0]:
# ================================================================
# Configurações
# ================================================================
catalog = "amazonia_catalog"
schema = "b_inep"
table_name = "d_fire_risk"

path_raw = "/Volumes/amazonia_catalog/raw/raw_inpe"
file_name = f"INPE_FireRiskModel_2.2_FireRisk_{data_ref_carga.replace('-', '')}.nc"
file_path = f"{path_raw}/{file_name}"

print(f"📂 Procurando arquivo: {file_path}")

In [0]:
#spark.read.format("binaryFile").option("pathGlobFilter", "*.nc").load("file:/Volumes/amazonia_catalog/raw/raw_inpe").inputFiles()

# ================================================================
# 1️⃣ Ler arquivo binário (.nc)
# ================================================================
# O formato 'binaryFile' lê qualquer arquivo binário, gerando colunas: path, modificationTime, length, content
try:
    df_bin = (
        spark.read.format("binaryFile")
        .load(file_path)
        .withColumn("data_ref_carga", F.lit(data_ref_carga))
        .withColumn("nome_arquivo", F.lit(file_name))
    )
except Exception as e:
    raise FileNotFoundError(f"❌ Erro ao ler o arquivo binário: {e}")

if df_bin.count() == 0:
    raise FileNotFoundError(f"⚠️ Nenhum arquivo encontrado no caminho informado: {file_path}")
else:
    print(f"✅ Arquivo lido com sucesso ({df_bin.count()} registro).")

df_bin.display(truncate=False)

In [0]:
try:
    df_bin = (
        spark.read.format("binaryFile")
        # O option("pathGlobFilter", "*.nc") é redundante, mas inofensivo
        .load(file_path)
        # O .inputFiles() estava sintaticamente incorreto aqui, foi removido.
        .withColumn("data_ref_carga", F.lit(data_ref_carga))
        .withColumn("nome_arquivo", F.lit(file_name))
    )
except Exception as e:
    # Capturando a exceção de forma mais específica para FileNotFoundError,
    # caso o caminho esteja certo, mas o arquivo não exista.
    if "No such file or directory" in str(e):
        raise FileNotFoundError(f"❌ Arquivo não encontrado no caminho: {file_path}")
    else:
        # Relança se for outro erro
        raise Exception(f"❌ Erro inesperado ao ler o arquivo: {e}")

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, BinaryType

In [0]:
# ================================================================
# 2️⃣ Quebra de Linhagem MANUAL (CRÍTICO)
# ================================================================

print("🛠️ Quebrando a linhagem do arquivo de origem...")

# 1. Coletar a única linha do DataFrame
# Isso é seguro porque df_bin tem apenas 1 linha. A falha no .toPandas()
# sugere um problema na *execução do plano de leitura*, mas a coleta simples
# é o único caminho restante. Se falhar aqui, o arquivo não está acessível.
try:
    row = df_bin.collect()[0]
except Exception as e:
    # Se a coleta falhar, o PATH_NOT_FOUND é o erro de leitura, e não de gravação.
    raise Exception(f"❌ Erro final na coleta da linha. O arquivo pode não estar acessível ao cluster no caminho: {file_path}. Detalhe: {e}")


# 2. Extrair o conteúdo e metadados
content_bytes = row["content"]
data_ref_carga_val = row["data_ref_carga"]
nome_arquivo_val = row["nome_arquivo"]

# 3. Definir o esquema final (sem as colunas problemáticas: path, length, modTime)
final_schema = StructType([
    StructField("content", BinaryType(), True),
    StructField("data_ref_carga", StringType(), True),
    StructField("nome_arquivo", StringType(), True)
])

# 4. Criar um novo DataFrame PySpark do ZERO, sem linhagem
df_para_gravar = spark.createDataFrame(
    [
        (content_bytes, data_ref_carga_val, nome_arquivo_val)
    ],
    schema=final_schema
)

In [0]:
# ================================================================
# 2️⃣ Gravar em tabela Delta particionada
# ================================================================
# 3. Executar a Gravação Delta
(
    df_para_gravar.write
    .format("delta")
    .mode("append")
    .partitionBy("data_ref_carga")
    .saveAsTable(full_table_name)
)

print(f"✅ Tabela Bronze criada/atualizada: {full_table_name}")